In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_2101157/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.7)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_7_29,0.903381,0.474133,0.902370,0.699974,0.788847,0.231038,3.516475,0.094934,0.435318,0.265126,1.042667,0.480664,1.358060,0.488355,162.930344,335.640991,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
1,model_1_7_28,0.903359,0.474395,0.904724,0.711419,0.796372,0.231090,3.514726,0.092645,0.418711,0.255678,1.046340,0.480718,1.358140,0.488410,162.929898,335.640545,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
2,model_1_7_30,0.903350,0.473860,0.899887,0.688328,0.781157,0.231111,3.518303,0.097349,0.452215,0.274782,1.039069,0.480740,1.358173,0.488432,162.929715,335.640362,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
3,model_1_7_27,0.903286,0.474643,0.906955,0.722673,0.803738,0.231264,3.513066,0.090475,0.402383,0.246429,1.050093,0.480900,1.358411,0.488594,162.928387,335.639034,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
4,model_1_7_31,0.903270,0.473573,0.897276,0.676505,0.773315,0.231302,3.520217,0.099888,0.469368,0.284628,1.035563,0.480939,1.358469,0.488634,162.928060,335.638707,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,model_1_2_4,0.807795,0.374959,0.968000,-0.031877,0.455513,0.459604,4.179655,0.035484,1.228155,0.631820,2.473853,0.677941,1.712289,0.688789,161.554781,334.265428,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
394,model_1_2_3,0.803921,0.371812,0.965366,-0.041991,0.449068,0.468868,4.200696,0.038404,1.240194,0.639299,2.458642,0.684739,1.726646,0.695696,161.514869,334.225516,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
395,model_1_2_2,0.800546,0.369411,0.962261,-0.053283,0.441793,0.476938,4.216749,0.041848,1.253634,0.647741,2.445969,0.690607,1.739153,0.701657,161.480737,334.191384,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
396,model_1_2_1,0.797045,0.366915,0.958760,-0.065074,0.434073,0.485309,4.233444,0.045730,1.267668,0.656699,2.432358,0.696641,1.752127,0.707788,161.445938,334.156585,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
